In [1]:
import numpy as np
# constant size
size = 10000
# Set p(H) apriori for simulation as a biased coin.
p_h = 0.7

# Model experiment as a single biased coin flipped 1000 times.
data = np.random.binomial(1, p_h, size=size)

In [2]:
print(data.shape)

(10000,)


In [4]:
n_states = 2
n_obs = data.shape[0]

emission = np.random.rand((n_obs))
emission[-1] = 1
transition = np.random.rand(n_states,n_states)